## Common part

In [ ]:
!pip install -q gwpy

In [ ]:
# == Mount google colab folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# INPUT_VIDEO = "/content/drive/MyDrive/kaggle/datasets/dfl-bundesliga-data-shootout/train/1606b0e6_0.mp4"
INPUT_VIDEO = "/content/drive/MyDrive/1606b0e6_0.mp4"
TEMP_DIR = "/content/tmp_video"
OUTPUT_DIR = "/content/clip_video"
input_video_fn = os.path.basename(INPUT_VIDEO)
input_video_name = input_video_fn.split('.')[0]

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## Use bytetrack for given video

In [ ]:
%%capture
# == Download the repo content and install dependencies ==
!git clone https://github.com/ifzhang/ByteTrack.git
%cd /content/ByteTrack/
%mkdir pretrained
%cd pretrained

# == Download pretrained X model weights ==
!gdown --id "1P4mY0Yyd3PPTybgZkjMYhFri88nTmJX5"
!gdown --id "11Zb0NN_Uu7JwUd9e6Nk8o2_EUfxWqsun"
!gdown --id "1uSmhXzyV1Zvb4TJJCzpsZOIcw7CCJLxj"

In [ ]:
%%capture
# == Install dependencies ==
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

%cd /content/ByteTrack/
!pip3 install -r requirements.txt

In [ ]:
%%capture
# == Install ByteTrack ==
!python3 setup.py develop

In [ ]:
# run inference demo (can be slow on colab). The cell output is deflected to the file 'log.txt' for downstream use to display the result but feel free to remove 
%cd /content/ByteTrack
!python3 tools/demo_track.py video \
         --path $INPUT_VIDEO \
         -f exps/example/mot/yolox_m_mix_det.py \
         -c pretrained/bytetrack_m_mot17.pth.tar \
         --fp16 --fuse --save_result &> log.txt

In [ ]:
# == Get rendered result video file path ==
import re
%cd /content/ByteTrack
with open('log.txt', 'r') as file:
    text = file.read().replace('\n', '')

m = re.search('video save_path is ./(.+?).mp4', text)
if m:
    found = '/content/ByteTrack/' + m.group(1) + ".mp4"
found

In [ ]:
import shutil
shutil.copy(found, TEMP_DIR)

In [ ]:
TMP_VIDEO = f"{TEMP_DIR}/{input_video_fn}"

In [ ]:
!cp /content/drive/MyDrive/tracking/1606b0e6_0.mp4 /content/tmp_video/1606b0e6_0.mp4

In [ ]:
TMP_VIDEO

## Cut video by given timestamp 

In [ ]:
!pip install minio
!pip install imageio_ffmpeg

In [ ]:
import uuid
import time
import pandas as pd
from tqdm import tqdm

import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from minio import Minio
from minio.error import S3Error

import psycopg2
from psycopg2 import OperationalError


In [ ]:
MARKUP_FILE = "/content/drive/MyDrive/kaggle/datasets/dfl-bundesliga-data-shootout/train.csv"
TIME_WINDOW = 5

### Minio Setting

In [ ]:
MINIO_ENDPOINT_URL = 
MINIO_ACCESS_KEY = 
MINIO_SECRET_KEY = 
MINIO_BUCKET =

client = Minio(
        MINIO_ENDPOINT_URL,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
)

def add_file_to_s3(input_path, object_name):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fput_object(MINIO_BUCKET, object_name, input_path)

def get_file_to_s3(object_name, file_path):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fget_object(MINIO_BUCKET, object_name, file_path)

### Postgres Setting

In [ ]:
POSTGRES_USER=
POSTGRES_PASSWORD=
POSTGRES_DB=
POSTGRES_HOST=
POSTGRES_PORT=

def pg_create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(database=db_name,
                                      user=db_user,
                                      password=db_password,
                                      host=db_host,
                                      port=db_port,)
        connection.autocommit = True
    except OperationalError as ex:
        print(f"The ERROR {ex} occurred")
    return connection

connection = pg_create_connection(POSTGRES_DB, 
                                  POSTGRES_USER, 
                                  POSTGRES_PASSWORD, 
                                  POSTGRES_HOST,
                                  POSTGRES_PORT)

def pg_insert_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_one_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            hander = cursor.fetchone()[0]
            return hander
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_many_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            return cursor.fetchall()
        except OperationalError as ex:
          print(f"The ERROR {ex} occurred")

def add_clip_to_db(video_id, action_type, stime, clip_path):
  action_id = pg_one_select_query(query=f"""
                                  SELECT id FROM public.action_types
                                  WHERE name=%s;
                                  """, param=(action_type,))
  query = f"""
      INSERT INTO events(id_video, id_action, time, clip_path)
      VALUES (%s, %s, %s, %s)
      RETURNING id;
  """
  pg_insert_query(query=query,
                  param=(video_id, action_id, stime, clip_path))

### Get preview from video

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
query_video_id = pg_many_select_query(query=f"""
                                SELECT id FROM public.videos
                                WHERE name=%s;
                                 """, param=(input_video_name,))
if len(query_video_id):
  video_id = query_video_id[0][0]
else:
  # Get preview
  cap = cv2.VideoCapture(INPUT_VIDEO)
  cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
  res, frame = cap.read()
  suuid = str(uuid.uuid4())
  filename_preview = f"{TEMP_DIR}/{str(uuid.uuid4())}.jpg"
  cv2.imwrite(filename_preview, frame)
  
  # Add to dp and s3
  add_file_to_s3(filename_preview, f'{input_video_name}/{suuid}.jpg')
  video_id = pg_one_select_query(query=f"""
                                INSERT INTO videos(name, preview_path) 
                                VALUES (%s)
                                RETURNING id;
                                """, param=(input_video_name, f'{input_video_name}/{suuid}.jpg'))
  # Remove tmp file
  os.remove(filename_preview)

### Main processing

In [ ]:
markup = pd.read_csv(MARKUP_FILE)
video_markup = markup[~(markup.event.isin(['start', 'end']))]
video_markup = video_markup[video_markup.video_id == input_video_name]
video_markup = video_markup[video_markup.time < 900]

In [ ]:
video_markup

In [ ]:
for index, row in tqdm(video_markup.iterrows()):
  timestamp = row["time"]
  action = row["event"]
  # Calc time for subclip
  start_t = max(timestamp - TIME_WINDOW, 0)
  # TODO: Find video time duration 
  end_t = timestamp + TIME_WINDOW

  suuid = str(uuid.uuid4())
  filename_mp4 = f"{OUTPUT_DIR}/{suuid}.mp4"
  filename_h264 = f"{OUTPUT_DIR}/{suuid}.h264"

  # Extract subclip
  cut_command = f"ffmpeg -ss {start_t} -i {TMP_VIDEO} -t {2*TIME_WINDOW} -async 1 {filename_mp4}"
  process = subprocess.run(cut_command.split(), stdout=subprocess.PIPE)

  # # Extract subclip
  # cut_command = f"ffmpeg -i {filename_mp4} -an -vcodec libx264 -crf 23 {filename_h264}"
  # process = subprocess.run(cut_command.split(), stdout=subprocess.PIPE)

  add_file_to_s3(filename_mp4, f'{input_video_name}/{suuid}.mp4')
  add_clip_to_db(video_id, action, timestamp, f'{input_video_name}/{suuid}.mp4')

  os.remove(filename_mp4)
  # os.remove(filename_h264)

In [ ]:
!rm -rf /content/clip_video/*

In [ ]:
add_file_to_s3("/content/ad6b7876-caed-41cc-bcd4-d970fa0de02f.jpg", f"{input_video_name}/ad6b7876-caed-41cc-bcd4-d970fa0de02f.jpg")